# First pass at new raster tutorial material


### Note: This notebook will focus on accessing Landsat8 and Sentinel2 public datasets on AWS (10-30m postings)

Target area for Cyclone Kenneth 2019-04-25
(building off https://github.com/shaystrong/sagely, https://medium.com/@shay.strong/openstreetmap-data-to-ml-training-labels-for-object-detection-ce0a3ccdb907, https://github.com/shaystrong/sagely/blob/master/osm_ml_training_pt1.ipynb) 

Goals identified by Friederich:
- pull in open source raster data on the fly for the specified bounding box, at a resolution sufficient to delineate buildings
- learn how to explore the data using rasterio and gdal etc.
- groom and clean - ready for ML (could be as simple as changing the projection and dropping unnecessary bands, for example)
- if open raster data sources aren’t at a high enough resolution - good discussion point - can stage a dataset that is. this would also teach folks how to add in their own data and coregister it into the same coordinate system and projection.


bbox = [44.158172607421875, -12.058123089867408, 44.553680419921875, -12.397682695006017]

In [ ]:
import matplotlib.pyplot as plt
# Might still be some issues with jupyterlab 1
#%matplotlib widget
%matplotlib inline

In [ ]:
bbox = [44.158172607421875, -12.058123089867408, 44.553680419921875, -12.397682695006017]
west, north, east, south = bbox
bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

## 1) Look at a leaflet slippy map to get accustomed to area of interest

In [ ]:
import ipyleaflet
from ipyleaflet import Map, Rectangle
m = Map(center=bbox_ctr, zoom=6)
rectangle = Rectangle(bounds=((south, west), (north, east))) #SW and NE corners of the rectangle (lat, lon)
m.add_layer(rectangle)
m

## 2) NASA GIBS provides free imagery tiles that can be easily displayed with ipyleaflet

The NASA Worldview web application is a way to explore all GIBS datasets https://worldview.earthdata.nasa.gov/
More here: https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/gibs

NOTES: 
* add some information about MODIS. Coarse resolution, not optimal for building detection, but daily and free!
* also, odds are there are clouds obstructing your view of earth's surface!
* GIBS has pre-rendered images good for visualizations, if you're interested in analytic values for scientific analysis, this isn't the best option
* Apparently the tiles only exist to a zoom level 9 (b/c very coarse resolution)

In [ ]:
from ipyleaflet import basemaps, basemap_to_tiles, TileLayer, SplitMapControl, Polygon

m = Map(center=bbox_ctr, zoom=8)# MODIS great for large areas (onl)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2019-06-30")
left_layer = TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m.add_layer(rectangle)

m

In [ ]:
# Putting it all together
import ipywidgets
import datetime
date_picker = ipywidgets.DatePicker(description='MODIS Image Date: ', value=datetime.datetime.today(), style=dict(description_width='initial'))
m = Map(center=bbox_ctr, zoom=8)# MODIS great for large areas (onl)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, )
left_layer = TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m.add_layer(rectangle)

def on_value_change(change):
    newurl = f'https://gibs.earthdata.nasa.gov/wmts/epsg3857/best/MODIS_Terra_CorrectedReflectance_TrueColor/default/{date}/GoogleMapsCompatible_Level9/{{z}}/{{y}}/{{x}}.jpg'
    right_layer.set_trait('url', newurl)

date_picker.observe(on_value_change)

display(date_picker, m)

## 3) Sat-search is a tool to discover free imagery on AWS 

https://github.com/sat-utils/sat-search
https://github.com/sat-utils/sat-stac

https://github.com/pangeo-data/intake-stac

https://registry.opendata.aws/
    
    
NOTES: 
* bbox and coordinate listing often have different orderings (lon, lat) (south, north, west, east)
* add a table comparing Landsat8, Sentinel2, CBERS (resolution/posting, repeat interval)

In [ ]:
# bbox as a python list is great for use in python, but we can instead save to a more interoperable format (GeoJSON)
# Here is a great website for creating and visualizing geojson on a map: http://geojson.io
import json

aoi = { "type": "Polygon", 
    "coordinates": [[[west, south], [west, north], [east, north], [east, south], [west, south]]]
}
# pretty print formatting
print(json.dumps(aoi, sort_keys=False, indent=2))

# save to file for future use
with open('aoi-5977.geojson', 'w') as f:
    json.dump(aoi, f)

In [ ]:
import geopandas as gpd

# Load results to pandas geodataframe
# now other packages such as geojson can read this file
gfa = gpd.read_file('aoi.json')
gfa

In [ ]:
# Sat-search provides an intuitive way to search for images using STAC catalogs (https://stacspec.org/)
import satsearch

results = satsearch.Search(bbox=bbox, datetime='2019-02-01/2019-06-01')
print('%s items' % results.found())
items = results.items()
print('%s collections:' % len(items._collections))
print(items._collections)


In [ ]:
# If you are unfamiliar with one of these satellites, we can look at stored metadata
col = items._collections[0]

print('Title:', col.title)
print('Collection Version:', col.version)
print('Keywords: ', col.keywords)
print('License:', col.license)
print('Providers:', col.providers)
print('Extent', col.extent)

In [ ]:
# We can delve deeper to see what kind of metadata is available at the scene level
for key in col.properties:
    if key == 'eo:bands':
        [print(band) for band in col[key]]
    else:
        print('%s: %s' % (key, col[key]))

In [ ]:
# How does Sentinel-2 data differ?
col = items._collections[1]
print('Title:', col.title)
print('Collection Version:', col.version)
print('Keywords: ', col.keywords)
print('License:', col.license)
print('Providers:', col.providers)
print('Extent', col.extent)

# We can delve deeper to see what kind of metadata is available at the scene level
for key in col.properties:
    if key == 'eo:bands':
        [print(band) for band in col[key]]
    else:
        print('%s: %s' % (key, col[key]))

In [ ]:
# Custom syntax (additional fields, query strings instead of query dict)
properties =  ["landsat:tier=T1"] 

bbox = (west, south, east, north) #(min lon, min lat, max lon, max lat)

results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

In [ ]:
# Save for later/sharing with others
from satstac import Items

items = results.items()
items.save('items-landsat8.json')
items = Items.load('items-landsat8.json')
#items.bbox()

In [ ]:
gfl = gpd.read_file('items-landsat8.json')
gfl = gfl.sort_values('datetime').reset_index(drop=True)
print('records:', len(gfl))
gfl.head()

In [ ]:
gfl['landsat:revision'].unique()

In [ ]:
# Are all datetimes unique? we have varying 'archive numbers GN02, GN03' https://gisgeography.com/landsat-file-naming-convention/
len(gfl['datetime'].unique())

In [ ]:
# Note the cloud_cover column, we can narrow our search by any of these propoerties
properties.extend(["eo:cloud_cover<10"])

test = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % test.found())

In [ ]:
# Or since we can just use geopandas to filter results
subset = gfl[gfl['eo:cloud_cover'] < 10]
print('%s items' % len(subset))

In [ ]:
# Plot search AOI and frames on a map
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

# just keep id for hover tips
cols = gfl.loc[:,('id','geometry')]

footprints = cols.hvplot(geo=True, line_color='k', alpha=0.1, title='Landsat 8 T1')
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) #.redim.range(Latitude=(45, 50), Longitude=(-126,-120)) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * aoi * labels

In [ ]:
# Pick the best path-row for aoi and browse thumbnails row=068 col=162
from ipywidgets import interact
from IPython.display import display, Image

def browse_images(items):
    n = len(items)

    def view_image(i=0):
        item = items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item['eo:cloud_cover']}")
        display(Image(item.asset('thumbnail')['href']))
    
    interact(view_image, i=(0,n-1))

In [ ]:
# Custom syntax (additional fields, query strings instead of query dict)
properties =  ["eo:row=068",
               "eo:column=162",
               "landsat:tier=T1"] 
results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())
items = results.items()

In [ ]:
# a good one LC81620682014200LGN01
browse_images(items) #Not updating... issue w/ ipywidgets jlab 1?

In [ ]:
# Same visualization for sentinel-2
# Custom syntax (additional fields, query strings instead of query dict)

# Also available in many locations https://www.usgs.gov/centers/eros/science/usgs-eros-archive-sentinel-2
# https://sentinel.esa.int/web/sentinel/sentinel-data-access
# https://gdal.org/drivers/raster/sentinel2.html

# remember we are searching data on AWS:
#https://registry.opendata.aws/sentinel-2/ 
properties = []
bbox = (west, south, east, north) #(min lon, min lat, max lon, max lat)

results = satsearch.Search.search(collection='sentinel-2-l1c', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

In [ ]:
items = results.items()
items.save('items-sentinel2.json')
#items = Items.load('items-sentinel2.json')
#items.bbox()

In [ ]:
gfs = gpd.read_file('items-sentinel2.json')
gfs = gfs.sort_values('datetime').reset_index(drop=True)
print('records:', len(gfs))
gfs.head()

In [ ]:
# Naming convention: https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention
# Comprehensive user guide: https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi

print(len(gfs['datetime'].unique()))
print(gfs['sentinel:sequence'].unique())

In [ ]:
gfs.datetime.head() # NOTE: probably a bug that datetime doesn't contain seconds information

In [ ]:
print(gfs['sentinel:grid_square'].unique())
print(gfs['sentinel:latitude_band'].unique())

In [ ]:
# just keep id for hover tips
cols = gfs.loc[:,('id','geometry')]

footprints = cols.hvplot(geo=True, line_color='k', alpha=0.1, title='Sentinel-2 L1C')
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) #.redim.range(Latitude=(45, 50), Longitude=(-126,-120)) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * aoi * labels

In [ ]:
# Seems there is some swath data in there in addition to military tiles or the polygon is incorrect...
# Actually, this is just a feature of converting swath data to standard tiles, you end up with a lot of nans
# NOTE: might also mention difference in "standard processing levels and compare to Landsat ARD data"


# Pull full resolution single-band analytic data

The rasterio library is great for working with georeferenced imagery in python

xarray is an excellent library for labelled multidimensional images

In [ ]:
import rasterio
import xarray as xr

In [ ]:
# Landsat first
# 48 PWC is our best tile

# Custom syntax (additional fields, query strings instead of query dict) row=068 col=162
properties =  ["eo:row=068",
               "eo:column=162",
               "landsat:tier=T1",
               "eo:cloud_cover<10"] 
results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())
items = results.items()

In [ ]:
browse_images(items)

In [ ]:
# These are environmnent variable settings for efficiently reading data on AWS S3
env = rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                 )

In [ ]:
# NOTE: should rename band to 'red' or '4'
item = items[0]
band = 'red'
url = item.asset(band)['href']
print(url)
with env:
    with rasterio.open(url) as src:
        width = src.width
        blockx = src.profile['blockxsize']
        blocky = src.profile['blockysize']
        #print(src.profile)
        xchunk = int(width/blockx)*blockx
        ychunk = blocky
        da = xr.open_rasterio(src, chunks={'band': 1, 'x': xchunk, 'y': ychunk})
da

In [ ]:
# This will pull raster data over network. if operating in the same region, should be very fast!
import hvplot.xarray

img = da.hvplot.image(rasterize=True, logz=True, width=700, height=500, cmap='reds', title=f'{item.id} ({band})')

img 

In [ ]:
# NOTE: UTM coords, if we want to overlay other things we need to convert coords
# NOTE: strange that CRS is UTM North rather than south...
# NOTE: would be nice to have common crs representation (currently using both gf.to_crs and ccrs)
from cartopy import crs as ccrs
crs = ccrs.UTM(zone='38N') 
img = da.hvplot.image(crs=crs, rasterize=True, width=700, height=500, cmap='reds', alpha=0.8, title=f'{item.id} ({band})') # , logz=True not working 
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles * img * aoi

In [ ]:
# NOTE: to get full resolution RGB images, need to merge separate R,G,B bands
#https://github.com/mapbox/landsat-tiler
#https://github.com/cogeotiff/rio-tiler #no public endpoint for these i think.
# public endpoint for tiling service that can handle RGB http://tiles.rdnt.io/

In [ ]:
print(gfa)
print(gfa.geometry.values)

In [ ]:
# Often we are only interested in small regions of full images. One of the killer features of cloud-optimized data formats stored
# on the cloud is that we can efficiently pull subsets of an image rather than the whole thing:
# https://github.com/mapbox/rasterio/blob/master/docs/topics/masking-by-shapefile.rst
# The projections of the shapefile and image need to match
import rasterio.mask

with rasterio.open(url) as src:
    # re-project vector to match raster CRS
    print(src.meta)
    shape = gfa.to_crs(epsg=src.crs.to_epsg())
    out_image, out_transform = rasterio.mask.mask(src, shape.geometry.values, crop=True)
    out_meta = src.meta
    out_meta.update({
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    print(out_meta)


In [ ]:
# Use a log scale
from matplotlib.colors import LogNorm

plt.imshow(out_image[0], cmap='Reds', norm=LogNorm())
plt.colorbar()

In [ ]:
# Use the highest resolution (15m pan band)
# NOTE: should rename band to 'red' or '4'
# NOTE: strange the Landsat is UTM *north* crs, sentinel2 is UTM *south*
item = items[0]
band = 'pan'
url = item.asset(band)['href']
print(url)
with env:
    with rasterio.open(url) as src:
        width = src.width
        blockx = src.profile['blockxsize']
        blocky = src.profile['blockysize']
        #print(src.profile)
        xchunk = int(width/blockx)*blockx
        ychunk = blocky
        da = xr.open_rasterio(src, chunks={'band': 1, 'x': xchunk, 'y': ychunk})
da

In [ ]:
# NOTE: UTM coords, if we want to overlay other things we need to convert coords to WGS84 or WebMercator
from cartopy import crs as ccrs
crs = ccrs.UTM(zone='38N') 
img = da.hvplot.image(crs=crs, rasterize=True, width=700, height=500, cmap='reds', alpha=0.8, title=f'{item.id} ({band})') # , logz=True not working 
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles * img * aoi

In [ ]:
# 48 PWC is our best tile
# Custom syntax (additional fields, query strings instead of query dict) 
properties =  ["sentinel:latitude_band=L",
               "sentinel:grid_square=MM",
               "eo:cloud_cover<10"] 
results = satsearch.Search.search(collection='sentinel-2-l1c', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())
items = results.items()

In [ ]:
def browse_images(items):
    n = len(items)

    def view_image(i=0):
        item = items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item['eo:cloud_cover']}")
        display(Image(item.asset('thumbnail')['href']))
    
    interact(view_image, i=(0,n-1))

In [ ]:
browse_images(items)

In [ ]:
# Plot single band, full resolution
# NOTE: sentinel2 is "requester pays", meaning we need AWS credentials, 
# or we need to be in the same region as the data (eu-central-1) and then there are no charges for access
# https://forum.sentinel-hub.com/t/how-to-access-s-2-meta-data-in-aws-after-bucket-goes-to-requester-pays/328
item = items[5]
print(item.assets.keys())
print(item.assets_by_common_name.keys())
print(item.asset('thumbnail')['href'])
print(item.asset('red')['href'])

In [ ]:
from rasterio.session import AWSSession
env = rasterio.Env(AWSSession(region_name='eu-central-1', 
                              requester_pays=True),
                  )
url = 's3://sentinel-s2-l1c/tiles/38/L/MM/2016/5/2/0/B04.jp2'
with env:
    with rasterio.open(url) as src:
        print(src.profile)

In [ ]:
band = 'red'
url = item.asset(band)['href']
# NOTE: need a function to turn https url (href) to s3:
url = 's3://sentinel-s2-l1c/tiles/38/L/MM/2016/5/2/0/B04.jp2'

from rasterio.session import AWSSession
env = rasterio.Env(AWSSession(region_name='eu-central-1', 
                              requester_pays=True),
                  )

with env:
    with rasterio.open(url) as src:
        width = src.width
        blockx = src.profile['blockxsize']
        blocky = src.profile['blockysize']
        #print(src.profile)
        xchunk = int(width/blockx)*blockx
        ychunk = blocky
        da = xr.open_rasterio(src, chunks={'band': 1, 'x': xchunk, 'y': ychunk})
da

In [ ]:
# NOTE: some sort of error happening here will need to investigate...

# NOTE: good example of all nans in area of interest
# These high-res images can take a while on local internet!
crs = ccrs.UTM(zone=38, southern_hemisphere=True) 
img = da.hvplot.image(crs=crs, rasterize=True, width=700, height=500, cmap='reds', alpha=0.8, title=f'{item.id} ({band})') # , logz=True not working 
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles * img * aoi